In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [3]:
tf.test.gpu_device_name()

''

In [8]:
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets),(X_test, y_test) = boston_housing.load_data()

In [9]:
print(train_data.shape)
print(X_test.shape)
print(train_targets[:10])
print(y_test[:10])

(404, 13)
(102, 13)
[15.2 42.3 50.  21.1 17.7 18.5 11.3 15.6 15.6 14.4]
[ 7.2 18.8 19.  27.  22.2 24.5 31.2 22.9 20.5 23.2]


In [10]:
# 표준화

mean = train_data.mean(axis = 0)
std = train_data.std(axis = 0)
train_data = train_data - mean
train_data = train_data / std

X_test = X_test - mean
X_test = X_test / std

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_data, train_targets,
test_size = 0.2,
random_state = 2045)

X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((323, 13), (81, 13), (323,), (81,))

In [12]:
from tensorflow.keras import models, layers

boston = models.Sequential(name = 'Regression')
boston.add(layers.Dense(64, activation = 'relu', input_shape = (13,)))
boston.add(layers.Dense(64, activation = 'relu'))
boston.add(layers.Dense(1))

In [13]:
boston.summary()

Model: "Regression"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                896       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________


In [14]:
boston.compile(loss = 'mse',
               optimizer = 'rmsprop',
               metrics = ['mae'])

In [16]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_mae',
                   mode = 'min',
                   patience = 50,
                   verbose = 1)

from tensorflow.keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint('best_boston.h5',
monitor = 'val_mae',
mode = 'min',
save_best_only = True,
verbose = 1)

In [17]:
%%time
Hist_boston = boston.fit(X_train, y_train,
                         epochs = 500,
                         batch_size = 1,
                         validation_data = (X_valid, y_valid),
                         callbacks = [es, mc],
                         verbose=1)

Epoch 1/500
323/323 [==============================] - 0s 1ms/step - loss: 4.1342 - mae: 1.3453 - val_loss: 10.3732 - val_mae: 2.2997

Epoch 00001: val_mae improved from inf to 2.29965, saving model to best_boston.h5
Epoch 2/500
323/323 [==============================] - 1s 2ms/step - loss: 4.0607 - mae: 1.3130 - val_loss: 13.0394 - val_mae: 2.4901

Epoch 00002: val_mae did not improve from 2.29965
Epoch 3/500
323/323 [==============================] - 1s 2ms/step - loss: 4.2808 - mae: 1.4199 - val_loss: 10.1619 - val_mae: 2.2385

Epoch 00003: val_mae improved from 2.29965 to 2.23855, saving model to best_boston.h5
Epoch 4/500
323/323 [==============================] - 0s 1ms/step - loss: 3.9659 - mae: 1.3511 - val_loss: 10.9096 - val_mae: 2.3294

Epoch 00004: val_mae did not improve from 2.23855
Epoch 5/500
323/323 [==============================] - 0s 1ms/step - loss: 3.9073 - mae: 1.3880 - val_loss: 9.8896 - val_mae: 2.2446

Epoch 00005: val_mae did not improve from 2.23855
Epoch 6/

In [20]:
!ls -l

total 76
-rw-r--r-- 1 root root 70280 Aug  5 08:21 best_boston.h5
drwxr-xr-x 1 root root  4096 Jul 16 13:20 sample_data


In [21]:
test_mse_score, test_mae_score = boston.evaluate(X_test, y_test)
print('MAE is :',test_mae_score)

4/4 [==============================] - 0s 3ms/step - loss: 13.9727 - mae: 2.4629
MAE is : 2.4628961086273193
